# SAVE PICKLES?


In [38]:
SAVING = False

# IMPORTING DATA

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import pandas as pd
import glob
import sys
sys.path.append('../')

# Make sure this import is after pandas
import Backend.Constants as cst
import Backend.NXCALS as nx



# Creating NXCALS variable containers
wires     = {'B1': [nx.NXCALSWire(loc = loc) for loc in ['L1B1','L5B1']],
             'B2': [nx.NXCALSWire(loc = loc) for loc in ['R1B2','R5B2']]}
beams     = [nx.NXCALSBeam(name) for name in ['B1','B2']]


# Locating data
DATA_PATH = '/eos/project/l/lhc-lumimod/LuminosityFollowUp/2022_test_v0.2.2/'
#DATA_PATH = '/home/lumimod/2022_07_26_NXCALS/2022_v3'
BMODE     = None#'STABLE'

# Importing fill info and wire current (~ 1 min import)
df = pd.read_parquet(DATA_PATH,
                     columns=['HX:FILLN','HX:BETASTAR_IP1'] + [wire.I for wire in wires['B1'] + wires['B2']] + [beam.Nb for beam in beams])


In [65]:
DATA_PATH = '/home/lumimod/2022_07_26_NXCALS/2022_v2'
df = pd.read_parquet(DATA_PATH+f'/HX:FILLN=8247/HX:BMODE=STABLE')
#display(df['RPMC.UL14.RBBCW.L1B1:I_MEAS'].dropna())
#display(df['RPMC.USC55.RBBCW.L5B1:I_MEAS'].dropna())


In [70]:
[k for k in df.keys() if 'BBCW' in k]

['RPMC.UL14.RBBCW.L1B1:I_MEAS']

In [47]:
[wire.I for wire in wires['B1'] + wires['B2']]

['RPMC.UL14.RBBCW.L1B1:I_MEAS',
 'RPMC.USC55.RBBCW.L5B1:I_MEAS',
 'RPMC.UL16.RBBCW.R1B2:I_MEAS',
 'RPMC.UL557.RBBCW.R5B2:I_MEAS']

In [64]:
display(df['RPMC.UL16.RBBCW.R1B2:I_MEAS'].dropna())
display(df['RPMC.UL557.RBBCW.R5B2:I_MEAS'].dropna())

KeyError: 'RPMC.UL16.RBBCW.R1B2:I_MEAS'

# Filtering: wires on  

In [39]:
filtered = df.loc[  #(df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Nb] > 900)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B1'] + wires['B2']] + [beam.Nb for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B1'] + wires['B2']]
relevant_idx    = results.index[np.any(results[current_culumns]>50,axis=1)]

results = results.loc[relevant_idx]

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("wires_on.pkl")  

,HX:BETASTAR_IP1,RPMC.UL14.RBBCW.L1B1:I_MEAS,RPMC.USC55.RBBCW.L5B1:I_MEAS,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BQM.B1:NO_BUNCHES,LHC.BQM.B2:NO_BUNCHES
HX:FILLN,,,,,,,
8033,30.0,350.029236,350.028625,350.026184,350.035034,1936.0,987.0
8063,30.0,350.030762,350.026855,350.029999,350.035187,987.0,987.0
8072,30.0,350.035919,350.028839,350.029205,350.030975,2463.0,1227.0
8076,30.0,350.032257,350.027771,350.026672,350.030304,1551.0,1551.0
8081,30.0,350.033661,350.028320,350.034973,350.033173,1551.0,1551.0
8083,30.0,350.037598,350.027008,350.030334,350.029175,1551.0,1551.0
8088,30.0,350.030609,350.026367,350.028961,350.044556,1935.0,1935.0
8094,30.0,350.029541,350.026764,350.029053,350.031921,1935.0,1935.0
8102,30.0,350.035645,350.028290,350.030029,350.043091,1935.0,1935.0


22 fills found


# Filtering: wires off

In [4]:
filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Nb] > 900)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B1'] + wires['B2']] + [beam.Nb for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B1'] + wires['B2']]
relevant_idx    = results.index[np.any(results[current_culumns]<50,axis=1)]

results = results.loc[relevant_idx]

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("wires_off.pkl")  

,HX:BETASTAR_IP1,RPMC.UL14.RBBCW.L1B1:I_MEAS,RPMC.USC55.RBBCW.L5B1:I_MEAS,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BQM.B1:NO_BUNCHES,LHC.BQM.B2:NO_BUNCHES
HX:FILLN,,,,,,,
8046,30.0,0.032104,0.026883,0.024001,0.032827,987.0,987.0
8067,30.0,0.028639,0.028914,0.023304,0.033751,1227.0,1227.0
8068,30.0,0.030998,0.028323,0.022394,0.032297,1227.0,1227.0
8073,30.0,0.027791,0.024217,0.016616,0.028722,1227.0,1227.0
8091,30.0,0.026838,0.029066,0.022227,0.031736,1935.0,1935.0
8106,30.0,0.032543,0.027489,0.019663,0.030994,1935.0,1935.0
8121,30.0,0.027352,0.025020,0.015586,0.028524,2413.0,2413.0
8147,30.0,0.029788,0.027777,0.019725,0.033933,2413.0,2413.0
8148,30.0,0.030818,0.030368,0.023622,0.033524,2413.0,2413.0


11 fills found


# Quality check: negative current?



In [5]:
filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Nb] > 900)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B1'] + wires['B2']] + [beam.Nb for beam in beams]].min().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B1'] + wires['B2']]
relevant_idx    = results.index[np.any(results[current_culumns]<-10,axis=1)]

results = results.loc[relevant_idx]

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("wires_neg.pkl")  

,HX:BETASTAR_IP1,RPMC.UL14.RBBCW.L1B1:I_MEAS,RPMC.USC55.RBBCW.L5B1:I_MEAS,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BQM.B1:NO_BUNCHES,LHC.BQM.B2:NO_BUNCHES
HX:FILLN,,,,,,,


0 fills found
